In [3]:
import requests

url = "https://api.themoviedb.org/3/movie/11?api_key=f5813332cb558d374cbcb057ea2fc48b"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"adult":false,"backdrop_path":"/4qCqAdHcNKeAHcK8tJ8wNJZa9cx.jpg","belongs_to_collection":{"id":10,"name":"Star Wars Collection","poster_path":"/r8Ph5MYXL04Qzu4QBbq2KjqwtkQ.jpg","backdrop_path":"/d8duYyyC9J5T825Hg7grmaabfxQ.jpg"},"budget":11000000,"genres":[{"id":12,"name":"Adventure"},{"id":28,"name":"Action"},{"id":878,"name":"Science Fiction"}],"homepage":"http://www.starwars.com/films/star-wars-episode-iv-a-new-hope","id":11,"imdb_id":"tt0076759","original_language":"en","original_title":"Star Wars","overview":"Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.","popularity":113.59,"poster_path":"/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg","production_companies":[{"id":1,"logo_path":"/tlVSws0RvvtPBwViUyOFAO0vcQS.png","name":"Lucasfilm

In [3]:
import requests
from bs4 import BeautifulSoup

link = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

r = requests.get(link)

soup = BeautifulSoup(r.content)

In [13]:
# In the wikipedia page there are multiple tables, the first one is selected (highest-grossing films)
table = soup.find_all("table", {"class": "wikitable"})[0]
rows = table.find_all("tr")

movies = []

for row in rows:
    # Print all text in italic
    if row.find("i"):
        movies.append(row.find("i").text)

movies

['Avatar',
 'Avengers: Endgame',
 'Avatar: The Way of Water',
 'Titanic',
 'Star Wars: The Force Awakens',
 'Avengers: Infinity War',
 'Spider-Man: No Way Home',
 'Jurassic World',
 'The Lion King',
 'The Avengers',
 'Furious 7',
 'Top Gun: Maverick',
 'Frozen II',
 'Barbie',
 'Avengers: Age of Ultron',
 'The Super Mario Bros. Movie',
 'Black Panther',
 'Harry Potter and the Deathly Hallows – Part 2',
 'Star Wars: The Last Jedi',
 'Jurassic World: Fallen Kingdom',
 'Frozen',
 'Beauty and the Beast',
 'Incredibles 2',
 'The Fate of the Furious',
 'Iron Man 3',
 'Minions',
 'Captain America: Civil War',
 'Aquaman',
 'The Lord of the Rings: The Return of the King',
 'Spider-Man: Far From Home',
 'Captain Marvel',
 'Transformers: Dark of the Moon',
 'Skyfall',
 'Transformers: Age of Extinction',
 'The Dark Knight Rises',
 'Joker',
 'Star Wars: The Rise of Skywalker',
 'Toy Story 4',
 'Toy Story 3',
 "Pirates of the Caribbean: Dead Man's Chest",
 'Rogue One: A Star Wars Story',
 'Aladdin',
